<a href="https://colab.research.google.com/github/NickSlm/ml1/blob/main/LSTM_Encoder_Decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil
import numpy as np
import zipfile, requests, io
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_text as tf_text
from tensorflow import keras
from pathlib import Path
from urllib.parse import urljoin
import tensorflow_hub as hub
import pandas as pd
import unicodedata
import re
assert tf.__version__ >= "2.0"

2023-12-03 10:53:58.252234: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 10:53:58.619254: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 10:53:58.619341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 10:53:58.697087: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 10:53:58.858918: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-03 10:53:58.860784: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
def unzip_file(filepath, target_dir):
    with zipfile.ZipFile(filepath, 'r') as zip_ref:
        zip_ref.extractall(target_dir)

def move_and_extract(zip_name, src_dir, dst_dir):
    if zip_name in os.listdir(dst_dir):
        unzip_file(os.path.join(dst_dir, zip_name), dst_dir)
        return Path(os.path.join(dst_dir, "heb.txt")).parent/"heb.txt"
        
    assert zip_name in os.listdir(src_dir)
    shutil.move(os.path.join(src_dir, zip_name), dst_dir)
    unzip_file(os.path.join(dst_dir, zip_name), dst_dir)
    return Path(os.path.join(dst_dir, "heb.txt")).parent/"heb.txt"
    
# def download_and_extract(zipname, url, dst_dir):
#     r = requests.get(url)
#     if r.status_code == 200:
#         print(r.status_code)
        # z = zipfile.ZipFile(io.BytesIO(r.content))
        # z.extractall(dst_dir)

In [3]:
ZIP_NAME = "heb-eng.zip"
dataset_dir = os.path.join("datasets", "sentences")
src_dir = os.path.abspath("/mnt/c/Users/nick/")

path_to_file = move_and_extract(ZIP_NAME, src_dir, dataset_dir)
# ZIP_NAME = "spa-eng.zip"
# download_url = "https://www.manythings.org/anki/spa-eng.zip"
# download_and_extract(ZIP_NAME, download_url, dataset_dir)

In [4]:
def load_data(path):
    text = path.read_text(encoding="utf-8")
    lines = text.splitlines()
    pairs = [line.split("\t") for line in lines]
    context = np.array([context for target, context, _ in pairs])
    target = np.array([target for target, context, _  in pairs])
    return target, context

In [5]:
def data_cleanup(data):
    """
    """
    return data

In [6]:
def create_dataset(raw_context, raw_target, buffer_size, batch_size=32):
    train_size = np.random.uniform(0, 1, buffer_size) < 0.8
    train_dataset = tf.data.Dataset.from_tensor_slices((raw_context[train_size], raw_target[train_size]))
    train_dataset = train_dataset.shuffle(buffer_size)
    train_dataset = train_dataset.batch(batch_size)

    test_dataset = tf.data.Dataset.from_tensor_slices((raw_context[~train_size], raw_target[~train_size]))
    test_dataset = test_dataset.shuffle(buffer_size)
    test_dataset = test_dataset.batch(batch_size)
    return train_dataset, test_dataset

In [7]:
def data_preprocess(text):
    text = tf_text.normalize_utf8(text, "NFKD")
    text = tf.strings.lower(text)
    text = tf.strings.regex_replace(text, '[^ a-zA-Zא-ת.?!,¿]', '')
    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
    text = tf.strings.strip(text)
    text = tf.strings.join(['[SOS]', text, '[EOS]'], separator=' ')
    return text

In [8]:
raw_context, raw_target = load_data(path_to_file)
train_dataset, test_dataset = create_dataset(raw_context, raw_target, len(raw_context) , batch_size=32)

for example_context_strings, example_target_strings in train_dataset.take(1):
    print(example_target_strings[:5])
    break

2023-12-03 10:54:03.596127: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:06:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-12-03 10:54:03.772089: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


tf.Tensor(
[b'\xd7\x99\xd7\xa9 \xd7\x9c\xd7\x96\xd7\x94 \xd7\xa2\xd7\xa8\xd7\x9a \xd7\xa2\xd7\xa6\xd7\x95\xd7\x9d.'
 b'\xd7\x90\xd7\xaa \xd7\x9e\xd7\x93\xd7\x91\xd7\xa8\xd7\xaa \xd7\xa2\xd7\x9c\xd7\x99\xd7\x99?'
 b'\xd7\x97\xd7\xa8\xd7\x98\xd7\x94 \xd7\x90\xd7\x99\xd7\xa0\xd7\x94 \xd7\x9e\xd7\x99\xd7\x9c\xd7\x94 \xd7\xa0\xd7\xa8\xd7\x93\xd7\xa4\xd7\xaa \xd7\x9c\xd7\x94\xd7\x91\xd7\xa2\xd7\xaa \xd7\xa6\xd7\xa2\xd7\xa8.'
 b'\xd7\x94\xd7\x9d \xd7\x90\xd7\x9b\xd7\x9c\xd7\x95 \xd7\x90\xd7\xa8\xd7\x95\xd7\x97\xd7\xaa \xd7\xa2\xd7\xa8\xd7\x91.'
 b'\xd7\x90\xd7\xa0\xd7\x99 \xd7\x9c\xd7\x90 \xd7\x97\xd7\x95\xd7\xa9\xd7\x91 \xd7\xa9\xd7\x90\xd7\xa0\xd7\x95 \xd7\x9e\xd7\xaa\xd7\x90\xd7\x99\xd7\x9e\xd7\x99\xd7\x9d \xd7\x90\xd7\x97\xd7\x93 \xd7\x9c\xd7\xa9\xd7\xa0\xd7\x99 \xd7\x99\xd7\x95\xd7\xaa\xd7\xa8.'], shape=(5,), dtype=string)


In [9]:
context_preprocess = tf.keras.layers.TextVectorization(
    standardize=data_preprocess,
    max_tokens=10000,
    ragged=True
)
target_preprocess = tf.keras.layers.TextVectorization(
    standardize=data_preprocess,
    max_tokens=10000,
    ragged=True
)

In [10]:
context_preprocess.adapt(train_dataset.map(lambda x, y: x))
target_preprocess.adapt(train_dataset.map(lambda x, y: y))

example_tokens = target_preprocess(example_target_strings)

target_vocab = np.array(target_preprocess.get_vocabulary())
tokens = target_vocab[example_tokens[4].numpy()]
print(' '.join(tokens))


[SOS] אני לא חושב שאנו מתאימים אחד לשני יותר . [EOS]


In [11]:
def process_text(context, target):
    context = context_preprocess(context).to_tensor()
    target = target_preprocess(target)
    target_in = target[:,:-1].to_tensor()
    target_out = target[:,1:].to_tensor()
    return (context, target_in), target_out

In [12]:
train_dataset = train_dataset.map(process_text, tf.data.AUTOTUNE)
test_dataset = test_dataset.map(process_text, tf.data.AUTOTUNE)

In [13]:
for (context, target_in), target_out in train_dataset.take(1):
    print(context[0])

tf.Tensor(
[   2   25  196   12   75 5748   21   36   75 1532    4    3    0    0
    0    0], shape=(16,), dtype=int64)


In [14]:
# encoder_input = keras.layers.Input(shape=[None], dtype=np.int32)
# decoder_input = keras.layers.Input(shape=[None], dtype=np.int32)
# sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

# embedding = keras.layers.Embedding(vocab_size, embed_size)
# encoder_embedding = embedding(encoder_input)
# decoder_embedding = embedding(decoder_input)

# encoder = keras.layers.LSTM(512, return_state=True)
# encoder_output, hidden_state, cell_state = encoder(encoder_embedding)
# encoder_state = [hidden_state, cell_state]

# sampler = tfa.seq2seq.sampler.TrainingSampler()

# decoder_cell = keras.layers.LSTMCell(512)
# output_layer = keras.layers.Dense(vocab_size)

# decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)

# final_outputs, final_state, final_sequence_lengths  = decoder(decoder_embedding, initial_state=encoder_state, sequence_length=sequence_lengths)
# Y_proba = tf.nn.softmax(final_outputs.rnn_output)
# model = keras.Model(inputs=[encoder_input, decoder_input, sequence_lengths],outputs=[Y_proba])